<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#Loading-the-Data-and-Required-modules" data-toc-modified-id="Loading-the-Data-and-Required-modules-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Loading the Data and Required modules</a></span></li><li><span><a href="#Cramer's-V-Coefficient-of-the-Features" data-toc-modified-id="Cramer's-V-Coefficient-of-the-Features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cramer's V Coefficient of the Features</a></span></li><li><span><a href="#Encoding-Categorical-Features" data-toc-modified-id="Encoding-Categorical-Features-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Encoding Categorical Features</a></span></li><li><span><a href="#Models'-Training-without-Undersampling" data-toc-modified-id="Models'-Training-without-Undersampling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Models' Training without Undersampling</a></span><ul class="toc-item"><li><span><a href="#One-Hot-Encoding" data-toc-modified-id="One-Hot-Encoding-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>One Hot Encoding</a></span></li><li><span><a href="#Binary-Encoding" data-toc-modified-id="Binary-Encoding-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Binary Encoding</a></span></li><li><span><a href="#Leave-One-Out-Encoding" data-toc-modified-id="Leave-One-Out-Encoding-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Leave-One-Out Encoding</a></span></li></ul></li><li><span><a href="#Models'-Training-with-Undersampling" data-toc-modified-id="Models'-Training-with-Undersampling-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Models' Training with Undersampling</a></span><ul class="toc-item"><li><span><a href="#Balanced-Random-Forest-Classifier" data-toc-modified-id="Balanced-Random-Forest-Classifier-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Balanced Random Forest Classifier</a></span></li><li><span><a href="#Training-with-Undersampling" data-toc-modified-id="Training-with-Undersampling-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Training with Undersampling</a></span><ul class="toc-item"><li><span><a href="#One-Hot-Encoding" data-toc-modified-id="One-Hot-Encoding-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>One Hot Encoding</a></span></li><li><span><a href="#Binary-Encoding" data-toc-modified-id="Binary-Encoding-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Binary Encoding</a></span></li><li><span><a href="#Leave-One-Out-Encoding" data-toc-modified-id="Leave-One-Out-Encoding-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Leave-One-Out Encoding</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-6.2.4"><span class="toc-item-num">6.2.4&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></li></ul></li><li><span><a href="#Fine-Tuning-and-Model-Selection" data-toc-modified-id="Fine-Tuning-and-Model-Selection-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Fine-Tuning and Model Selection</a></span><ul class="toc-item"><li><span><a href="#Performance-of-Selected-Model" data-toc-modified-id="Performance-of-Selected-Model-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Performance of Selected Model</a></span></li><li><span><a href="#Focus-on-the-Known-Primary-Contributory-Cause" data-toc-modified-id="Focus-on-the-Known-Primary-Contributory-Cause-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Focus on the Known Primary Contributory Cause</a></span></li></ul></li></ul></div>

#  Overview

After we have explored the data visually and statistically, we are now ready to build a predictive model from the features of crashes in order to predict the crash type: Injury or No Injury crash. In this report, we go over the steps done to build the predictive model. We address first the different encoding schemes we tried for categorical variables, we then train different models and compare their performance. Since we have imbalances in the data (around 78% for no Injury crashes and 22% for Injury crashes), we compute the performance of the trained models in terms of their precision, recall and F1 scores. Since the performance of the trained models can be affected by the imbalances in the data, we also consider undersampling for the No Injury crash type and train the models accordingly. We then choose the best model and compute its performance on the test set.  

# Loading the Data and Required modules

We import the required modules, and we load the crashes' features and type.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import chi2_contingency
import category_encoders as ce
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score as accuracy, precision_score as precision
from sklearn.metrics import recall_score as recall, f1_score as f1
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler

In [2]:
X = pd.read_csv("crash_features.csv")
Y = pd.read_csv("crash_type.csv")

We map the two crash types as follows:
- "INJURY AND / OR TOW DUE TO CRASH": 1
- "NO INJURY / DRIVE AWAY": 0

In [3]:
Y = (Y["CRASH_TYPE"] == "INJURY AND / OR TOW DUE TO CRASH").values.astype(np.int)

We split the data into two sets: 80% for training and 20% for testing. We use the training set to train multiple models, compare between their performance and select the final model using cross validation. On the other hand, we use the test set to test the final model and report its predicted performance.

In [4]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X.iloc[:, 1:], Y, train_size=0.8, random_state=48)

# Cramer's V Coefficient of the Features

Based on our previous data exploration, we noticed that some features have weak relationship with the crash type. To check again the association between the features and the crash type, we compute the Cramer's V coefficients for each feature (using the training set).

In [5]:
colnames = Xtrain.columns


def cram(chi2, table):
    ntot = table.sum().sum()
    nrow, ncol = table.shape
    phi_sqr = chi2/ntot
    phi_sqr_tilde = max(0, phi_sqr-(nrow-1)*(ncol-1)/(ntot-1))
    ncol_tilde = ncol-((ncol-1)**2)/((ntot-1))
    nrow_tilde = nrow-((nrow-1)**2)/((ntot-1))
    cramer = np.sqrt(phi_sqr_tilde/min(ncol_tilde-1, nrow_tilde-1))
    return cramer


# Initialize an empty array that will have the results for each feature
values = np.empty(shape=[1, len(colnames)])
col = 0

# Loop over the features and save cramer's v coefs
for colname in colnames:
    table = pd.crosstab(Ytrain, Xtrain[colname])
    chi2, p, dof, expected = chi2_contingency(table)
    cramer = cram(chi2, table)
    values[0][col] = cramer
    col = col + 1

# Create a dataframe for the results
results = pd.DataFrame(values)
results.columns = colnames
results.rename(index={0: "Cramer's V"}, inplace=True)
results.T

,Cramer's V
POSTED_SPEED_LIMIT,0.126719
TRAFFIC_CONTROL_DEVICE,0.119743
DEVICE_CONDITION,0.106610
WEATHER_CONDITION,0.056911
LIGHTING_CONDITION,0.131978
FIRST_CRASH_TYPE,0.355483
TRAFFICWAY_TYPE,0.165673
ALIGNMENT,0.055291
ROADWAY_SURFACE_COND,0.062530
PRIM_CONTRIBUTORY_CAUSE,0.305199


We notice that some crash features have low Cramer's V coefficients (crash_month: 0.023 and crash_day_of_week: 0.039). Although low values of Cramer's V coefficients reflect weak relationship between the feature and the crash type, we decide for now to keep all the features and to not remove any column, especially that we have only 14 features in total, while the number of samples in the training set is around 240k samples.

# Encoding Categorical Features

The data consists of two types of categorical features:
- nominal: which we need to convert to numerical features;
- ordinal: which are the speed limit, crash hour, crash month and crash day of the week; since these variables are already in numerical form, we do not modify them in the preprocessing step.

Before we start training the model, we convert the entries of the nominal categorical features into numerical entries. One of the basic encoding scheme is one-hot encoding, which replaces a categorical feature with a number of columns equal to the number of levels or categories within the considered feature. Each level is then mapped to a series of 0 and 1, where the corresponding column of the level takes 1 and the remaining columns take 0. One of the main drawback of this scheme is that it increases the number of features in the data. Another encoding scheme that can replace a given categorical feature with a less number of columns is binary encoding, where each level of a given categorical feature is mapped to a binary number. There are also other types of encoding that replace a categorical feature with only one column, where each level of the given feature is mapped to a number computed based on its frequency. Some other schemes embed the target variable in the computation. For instance, target encoding is an encoding scheme that replaces features with a combination of posterior probability of the target given particular categorical value and the prior probability of the target over all the training data. Leave-one-out is "very similar to target encoding but excludes the current row’s target when calculating the mean target for a level One of these schemes", which makes this scheme less prone to overfitting and response leakage. For reference, check [here](https://contrib.scikit-learn.org/categorical-encoding/leaveoneout.html) and [here](https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02) 

We next consider these different schemes of encoding, namely one hot, binary and leave-one-out encodings, and try them with multiple training models to choose at the end the best combination. We create two versions of the training set based on binary and one hot encoding.

In [6]:
import category_encoders as ce

# names of columns to encode
col_names = ['TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION',
             'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE',
             'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'PRIM_CONTRIBUTORY_CAUSE', 'AREAS']

# use binary encoding to encode two categorical features
encoder_bin = ce.BinaryEncoder(cols=col_names).fit(Xtrain)
encoder_one_hot = ce.OneHotEncoder(cols=col_names).fit(Xtrain)

# transform the dataset
Xtr_bin = encoder_bin.transform(Xtrain)
Xtr_one_hot = encoder_one_hot.transform(Xtrain)

We are also going to consider the leave-one-out encoding scheme, but since this encoding uses the target variable (type of crash) while encoding, we are going to encode the set of the data folds that we are going to train while doing cross-validation. It will become clearer in the sequel.

Before we proceed, we outline the next steps:
- For each encoding scheme, we train the following set of training models: logistic regression, linear support vector machine (for linear models), random forest, Ada Boost and gradient tree boosting (for tree based models) and multinomial Naive Bayes, keeping their default parameters. The performance of the trained models is computed in terms of F1-score using 5-fold cross validation. 
- We then address the imbalances in the data by considering the model: BalancedRandomForestClassifier from the imblearn library and by also doing undersampling for the No Injury crashes and then training the same set of regular models we mentioned in the previous step.
- We finally select the best combination of encoding and training models, we re-fit the final model using the whole training set and then test the performance of the final model using the testing set.

# Models' Training without Undersampling

We first train various classifiers while keeping the imbalanced data as is. We will try the three different encoding schemes and summarize at the end the obtained results in a table. We first define the functions that we are going to use to train the training set and report the performance by means of cross-validation. The use of cross-validation is important to avoid overfitting and to correctly compare the performance between the different trained models.

Let us now define two functions to use in our analysis. The first function takes as input the model to train, the features, the target variable and the number of folds for cross-validation. We are going to use this function with the data encoded using one-hot and binary encoding schemes. This function uses cross-validation to train the model of interest and returns the means of the following metrics: accuracy, precision, recall and F1-score. 

In [7]:
from sklearn.model_selection import KFold


def cv_score(clf, x, y, cv, scale=False):
    result = np.zeros((cv, 4))
    i = 0
    # split data into train/test groups, 'cv' times
    for train, test in StratifiedKFold(cv).split(x, y):
        x_train = x[train]
        x_test = x[test]
        if (scale == True):
            scaler = StandardScaler()
            x_train = scaler.fit_transform(x_train)
            x_test = scaler.transform(x_test)
        clf.fit(x_train, y[train])  # fit
        ypred = clf.predict(x_test)
        result[i, 0] = accuracy(ypred, y[test])
        result[i, 1] = precision(ypred, y[test])
        result[i, 2] = recall(ypred, y[test])
        result[i, 3] = f1(ypred, y[test])
        i = i+1
    return np.mean(result, axis=0)

Similarly to the previous function, this second function also performs cross-validation to train the desired model, but first encodes the folds that are intended for training using leave-one-out scheme.

In [8]:
def cv_score_LOO(clf, x, y, cv, colnames, scale=False):
    result = np.zeros((cv, 4))
    i = 0
    # split data into train/test groups, 'cv' times
    for train, test in StratifiedKFold(cv).split(x, y):
        encoder = ce.leave_one_out.LeaveOneOutEncoder(cols=colnames).fit(
            x.iloc[train, ], y[train])
        x_trans = encoder.transform(x.iloc[train])
        x_test = encoder.transform(x.iloc[test])
        if (scale == True):
            scaler = StandardScaler()
            x_trans = scaler.fit_transform(x_trans)
            x_test = scaler.transform(x_test)
        clf.fit(x_trans, y[train])  # fit
        ypred = clf.predict(x_test)
        result[i, 0] = accuracy(ypred, y[test])
        result[i, 1] = precision(ypred, y[test])
        result[i, 2] = recall(ypred, y[test])
        result[i, 3] = f1(ypred, y[test])
        i = i + 1
    return np.mean(result, axis=0)

##### Metrics Used
Note that we are not only returning the accuracy of the trained model, but also its precision, recall and F1-score. As we previously mentioned, the metric of accuracy won't be helpful because of the imbalances in the data, this is why we look at different metrics. Since we are interested in detecting the Injury crashes, the metric recall measures how well the model is able to detect the Injury crash (minority class) by computing the true positive rate, and the metric precision indicates how much of the returned 1 labels do actually belong to Injury crashes. F1 score is the harmonic mean between precision and recall, and it is the metric we are going to use in deciding on the final model.

We are now ready to train the models for each encoding scheme.

## One Hot Encoding

We start with the one hot encoding, and we use the classifiers with their default parameters.

In [9]:
logistic_reg = SGDClassifier(loss='log')
linear_svm = SGDClassifier()
random_forest = RandomForestClassifier()
ada_boost = AdaBoostClassifier()
grad_boosting = GradientBoostingClassifier()
naive_bayes = MultinomialNB()

We now train the models. Note that for linear models we made sure to standardize the features.

In [10]:
result_lg = cv_score(logistic_reg, Xtr_one_hot.values,
                     Ytrain, cv=5, scale=True)
result_lsvm = cv_score(linear_svm, Xtr_one_hot.values,
                       Ytrain, cv=5, scale=True)
result_rf = cv_score(random_forest, Xtr_one_hot.values, Ytrain, cv=5)
result_aboost = cv_score(ada_boost, Xtr_one_hot.values, Ytrain, cv=5)
result_gboost = cv_score(grad_boosting, Xtr_one_hot.values, Ytrain, cv=5)
result_nb = cv_score(naive_bayes, Xtr_one_hot.values, Ytrain, cv=5)

We group the result into the following dataframe.

In [11]:
results = pd.DataFrame(
    [result_nb, result_lg, result_lsvm, result_rf, result_aboost, result_gboost])
results.columns = ["Accuracy", "Precision", "Recall", "F1"]
results.rename(index={0: "Naive Bayes", 1: "Logistic Regression", 2: "Linear SVM",
                      3: "Random Forest", 4: "Ada Boost",
                      5: "Gradient Boosting"}, inplace=True)
results.index.name = "One Hot Encoding"
results

,Accuracy,Precision,Recall,F1
One Hot Encoding,,,,
Naive Bayes,0.798314,0.431709,0.562788,0.488599
Logistic Regression,0.814306,0.319160,0.679115,0.433918
Linear SVM,0.807241,0.251848,0.686132,0.368242
Random Forest,0.815023,0.353664,0.659661,0.460459
Ada Boost,0.816063,0.304723,0.702781,0.425097
Gradient Boosting,0.816224,0.263173,0.752400,0.389947


We see that for all of the models, the accuracy is around 80%, which is expected because of the imbalances in the data. Now if we look at the values of precision and recall, we notice that while some models have good recall, we see that that their precision is much lower. This is reflected by the F1-scores that are below 0.5.

## Binary Encoding

We now train the models with the features being binary encoded.

In [12]:
result_lg = cv_score(logistic_reg, Xtr_bin.values, Ytrain, cv=5, scale=True)
result_lsvm = cv_score(linear_svm, Xtr_bin.values, Ytrain, cv=5, scale=True)
result_rf = cv_score(random_forest, Xtr_bin.values, Ytrain, cv=5)
result_aboost = cv_score(ada_boost, Xtr_bin.values, Ytrain, cv=5)
result_gboost = cv_score(grad_boosting, Xtr_bin.values, Ytrain, cv=5)
result_nb = cv_score(naive_bayes, Xtr_bin.values, Ytrain, cv=5)

The results are grouped as follows.

In [13]:
results_bin = pd.DataFrame(
    [result_nb, result_lg, result_lsvm, result_rf, result_aboost, result_gboost])
results_bin.columns = ["Accuracy", "Precision", "Recall", "F1"]
results_bin.rename(index={0: "Naive Bayes", 1: "Logistic Regression", 2: "Linear SVM",
                          3: "Random Forest", 4: "Ada Boost",
                          5: "Gradient Boosting"}, inplace=True)
results_bin.index.name = "Binary Encoding"
results_bin

,Accuracy,Precision,Recall,F1
Binary Encoding,,,,
Naive Bayes,0.775947,0.188108,0.494881,0.272592
Logistic Regression,0.785885,0.160535,0.572644,0.249994
Linear SVM,0.776799,0.003309,0.500554,0.006567
Random Forest,0.811426,0.319868,0.659977,0.430882
Ada Boost,0.787542,0.162181,0.586988,0.254136
Gradient Boosting,0.810177,0.246975,0.716941,0.367374


We see that the models performed worse with the binary encoding. 

## Leave-One-Out Encoding

We now train the models with the features being encoded with the leave-one-out encoding scheme.

In [14]:
result_lg = cv_score_LOO(logistic_reg, Xtrain, Ytrain,
                         colnames=col_names, cv=5, scale=True)
result_lsvm = cv_score_LOO(linear_svm, Xtrain, Ytrain,
                           cv=5, colnames=col_names, scale=True)
result_rf = cv_score_LOO(random_forest, Xtrain, Ytrain,
                         colnames=col_names, cv=5)
result_aboost = cv_score_LOO(
    ada_boost, Xtrain, Ytrain, colnames=col_names, cv=5)
result_gboost = cv_score_LOO(
    grad_boosting, Xtrain, Ytrain, colnames=col_names, cv=5)
result_nb = cv_score_LOO(naive_bayes, Xtrain, Ytrain, colnames=col_names, cv=5)

The results are as follows.

In [15]:
results_loo = pd.DataFrame(
    [result_nb, result_lg, result_lsvm, result_rf, result_aboost, result_gboost])
results_loo.columns = ["Accuracy", "Precision", "Recall", "F1"]
results_loo.rename(index={0: "Naive Bayes", 1: "Logistic Regression",
                          2: "Linear SVM", 3: "Random Forest",
                          4: "Ada Boost", 5: "Gradient Boosting"}, inplace=True)
results_loo.index.name = "Leave-One-Out Encoding"
results_loo

,Accuracy,Precision,Recall,F1
Leave-One-Out Encoding,,,,
Naive Bayes,0.778478,0.011573,0.736858,0.022786
Logistic Regression,0.815354,0.314403,0.691126,0.431386
Linear SVM,0.811000,0.236390,0.745456,0.356384
Random Forest,0.815056,0.367837,0.651800,0.470277
Ada Boost,0.816426,0.320444,0.691529,0.437905
Gradient Boosting,0.819663,0.331687,0.703636,0.450841


We see that the ensemble methods (random forest, ada-boost and gradient boosting) have better F1-score with the leave-one-out encoding scheme than with the one-hot encoding. Logistic regression as well as linear SVM performed the same in terms of F1-score in both encoding scheme (one-hot and leave-one-out).

With the presence of imbalances in the data, we saw that the models did not perform very well in terms of F1-score. Let us now check if we balance the data and train the models with the rebalanced data, how the performance will be affected.

# Models' Training with Undersampling

To address the imbalances in the data, we are going to first use the balanced random Forest classifier model, which internally handles the imbalances in the data. We are also going to train the same set of models we trained previously, but after undersampling the data that belong to the majority class (No Injury crashes). We are choosing the undersampling technique because we have enough data to sample from.      

## Balanced Random Forest Classifier
We start with the balanced random forest classifier. We try it with both encoding schemes: one-hot and binary. 

In [16]:
balanced_rf = BalancedRandomForestClassifier()

In [17]:
result_balancedrf_onehot = cv_score(
    balanced_rf, Xtr_one_hot.values, Ytrain, cv=5)
result_balancedrf_bin = cv_score(balanced_rf, Xtr_bin.values, Ytrain, cv=5)

We obtain the following result.

In [18]:
results_blrf = pd.DataFrame([result_balancedrf_onehot, result_balancedrf_bin])
results_blrf.columns = ["Accuracy", "Precision", "Recall", "F1"]
results_blrf.rename(index={0: "One Hot", 1: "Binary"}, inplace=True)
results_blrf.index.name = "Balanced Random Forest"
results_blrf

,Accuracy,Precision,Recall,F1
Balanced Random Forest,,,,
One Hot,0.741213,0.693878,0.448465,0.544806
Binary,0.728644,0.696034,0.432888,0.533789


We notice that with this balanced version of random forest, the accuracy and recall scores dropped, but the precision and F1 scores both increased. 

## Training with Undersampling

We now consider undersampling for the No Injury crashes. We first redefine the cross validation functions to take into account undersampling. For each split of the training set, we first perform undersampling on the k-1 folds and then train the model on the balanced data and then test on the k-th fold. 

In [19]:
def cv_score_undersample(clf, x, y, cv, scale=False):
    result = np.zeros((cv, 4))
    i = 0
    rus = RandomUnderSampler(random_state=0)
    # split data into train/test groups, 'cv' times
    for train, test in StratifiedKFold(cv).split(x, y):
        x_train, y_train = rus.fit_resample(x[train], y[train])
        x_test = x[test]
        if (scale == True):
            scaler = StandardScaler()
            x_train = scaler.fit_transform(x_train)
            x_test = scaler.transform(x_test)
        clf.fit(x_train, y_train)  # fit
        ypred = clf.predict(x_test)
        result[i, 0] = accuracy(ypred, y[test])
        result[i, 1] = precision(ypred, y[test])
        result[i, 2] = recall(ypred, y[test])
        result[i, 3] = f1(ypred, y[test])
        i = i+1
    return np.mean(result, axis=0)

This second function takes into account undersampling and leave one-out encoding technique.

In [20]:
def cv_score_LOO_undersample(clf, x, y, cv, colnames, scale=False):
    result = np.zeros((cv, 4))
    i = 0
    rus = RandomUnderSampler(random_state=0)
    # split data into train/test groups, 'cv' times
    for train, test in StratifiedKFold(cv).split(x, y):
        x_train, y_train = rus.fit_resample(x.iloc[train], y[train])
        encoder = ce.leave_one_out.LeaveOneOutEncoder(cols=colnames).fit(
            x_train, y_train)
        x_trans = encoder.transform(x_train)
        x_test = encoder.transform(x.iloc[test])
        if (scale == True):
            scaler = StandardScaler()
            x_trans = scaler.fit_transform(x_trans)
            x_test = scaler.transform(x_test)
        clf.fit(x_trans, y_train)  # fit
        ypred = clf.predict(x_test)
        result[i, 0] = accuracy(ypred, y[test])
        result[i, 1] = precision(ypred, y[test])
        result[i, 2] = recall(ypred, y[test])
        result[i, 3] = f1(ypred, y[test])
        i = i + 1
    return np.mean(result, axis=0)

### One Hot Encoding
We start with the one hot encoding scheme.

In [21]:
result_lg = cv_score_undersample(
    logistic_reg, Xtr_one_hot.values, Ytrain, cv=5, scale=True)
result_lsvm = cv_score_undersample(
    linear_svm, Xtr_one_hot.values, Ytrain, cv=5, scale=True)
result_rf = cv_score_undersample(
    random_forest, Xtr_one_hot.values, Ytrain, cv=5)
result_aboost = cv_score_undersample(
    ada_boost, Xtr_one_hot.values, Ytrain, cv=5)
result_gboost = cv_score_undersample(
    grad_boosting, Xtr_one_hot.values, Ytrain, cv=5)
result_nb = cv_score_undersample(naive_bayes, Xtr_one_hot.values, Ytrain, cv=5)

We obtain the following results.

In [22]:
results = pd.DataFrame(
    [result_nb, result_lg, result_lsvm, result_rf, result_aboost, result_gboost])
results.columns = ["Accuracy", "Precision", "Recall", "F1"]
results.rename(index={0: "Naive Bayes", 1: "Logistic Regression",
                      2: "Linear SVM", 3: "Random Forest",
                      4: "Ada Boost", 5: "Gradient Boosting"}, inplace=True)
results.index.name = "One Hot Encoding - Undersampling"
results

,Accuracy,Precision,Recall,F1
One Hot Encoding - Undersampling,,,,
Naive Bayes,0.707431,0.683919,0.407411,0.510633
Logistic Regression,0.724030,0.676988,0.425774,0.522726
Linear SVM,0.732649,0.659868,0.434885,0.524229
Random Forest,0.724184,0.688775,0.426927,0.527120
Ada Boost,0.734721,0.671885,0.438476,0.530635
Gradient Boosting,0.733068,0.679161,0.436962,0.531776


### Binary Encoding
We now try undersmapling with the binary encoding scheme.

In [23]:
result_lg = cv_score_undersample(
    logistic_reg, Xtr_bin.values, Ytrain, cv=5, scale=True)
result_lsvm = cv_score_undersample(
    linear_svm, Xtr_bin.values, Ytrain, cv=5, scale=True)
result_rf = cv_score_undersample(random_forest, Xtr_bin.values, Ytrain, cv=5)
result_aboost = cv_score_undersample(ada_boost, Xtr_bin.values, Ytrain, cv=5)
result_gboost = cv_score_undersample(
    grad_boosting, Xtr_bin.values, Ytrain, cv=5)
result_nb = cv_score_undersample(naive_bayes, Xtr_bin.values, Ytrain, cv=5)

We obtain the following result.

In [24]:
results_bin = pd.DataFrame(
    [result_nb, result_lg, result_lsvm, result_rf, result_aboost, result_gboost])
results_bin.columns = ["Accuracy", "Precision", "Recall", "F1"]
results_bin.rename(index={0: "Naive Bayes", 1: "Logistic Regression",
                          2: "Linear SVM", 3: "Random Forest",
                          4: "Ada Boost", 5: "Gradient Boosting"}, inplace=True)
results_bin.index.name = "Binary Encoding - Undersampling"
results_bin

,Accuracy,Precision,Recall,F1
Binary Encoding - Undersampling,,,,
Naive Bayes,0.647071,0.620837,0.340553,0.439838
Logistic Regression,0.681158,0.635291,0.373948,0.470741
Linear SVM,0.682150,0.628064,0.373948,0.468630
Random Forest,0.716630,0.690158,0.418295,0.520882
Ada Boost,0.686125,0.662930,0.382711,0.485272
Gradient Boosting,0.719132,0.679721,0.420178,0.519312


### Leave-One-Out Encoding
We now try udnersampling with the leave-one-out encoding scheme.

In [25]:
result_lg = cv_score_LOO_undersample(
    logistic_reg, Xtrain, Ytrain, colnames=col_names, cv=5, scale=True)
result_lsvm = cv_score_LOO_undersample(linear_svm, Xtrain, Ytrain,
                                       cv=5, colnames=col_names, scale=True)
result_rf = cv_score_LOO_undersample(random_forest, Xtrain, Ytrain,
                                     colnames=col_names, cv=5)
result_aboost = cv_score_LOO_undersample(
    ada_boost, Xtrain, Ytrain, colnames=col_names, cv=5)
result_gboost = cv_score_LOO_undersample(
    grad_boosting, Xtrain, Ytrain, colnames=col_names, cv=5)
result_nb = cv_score_LOO_undersample(
    naive_bayes, Xtrain, Ytrain, colnames=col_names, cv=5)

We obtain the following result.

In [26]:
results_loo = pd.DataFrame(
    [result_nb, result_lg, result_lsvm, result_rf, result_aboost, result_gboost])
results_loo.columns = ["Accuracy", "Precision", "Recall", "F1"]
results_loo.rename(index={0: "Naive Bayes", 1: "Logistic Regression",
                          2: "Linear SVM", 3: "Random Forest",
                          4: "Ada Boost", 5: "Gradient Boosting"}, inplace=True)
results_loo.index.name = "Leave-One-Out Encoding - Undersampling"
results_loo

,Accuracy,Precision,Recall,F1
Leave-One-Out Encoding - Undersampling,,,,
Naive Bayes,0.581123,0.515828,0.270280,0.354703
Logistic Regression,0.724213,0.682684,0.426769,0.524932
Linear SVM,0.732649,0.667424,0.435784,0.527131
Random Forest,0.726102,0.686289,0.428996,0.527960
Ada Boost,0.734368,0.683738,0.438968,0.534666
Gradient Boosting,0.741069,0.685153,0.447673,0.541520


### Conclusion

We noticed that with undersampling the accuracy and recall scores decreased in general for all models and types of encoding, but the precision and F1 scores got better. We noticed that most of the models performed better with the one-hot and leave-one-out encodings than with the binary encodings. We also noticed that most of the models have very similar performance. This might suggest that the data might not have enough information that can help us in detecting crashes with Injury.

For further tuning, we choose the algorithms with highest F1-scores: gradient boosting algorithm with leave-one-out encoding and balanced random forest tree with one-hot encoding.

# Fine-Tuning and Model Selection

We now fine-tune the two chosen model to see if we can have a boost in the F1-score.

In [27]:
n_estimators = [300, 500, 600]
max_depth = [3, 4, 5]

max_score = 0
for n in n_estimators:
    for d in max_depth:
        clf = GradientBoostingClassifier(n_estimators=n, max_depth=d)
        result = cv_score_LOO_undersample(
            clf, Xtrain, Ytrain, colnames=col_names, cv=5)
        if (result[3] > max_score):
            max_score = result[3]
            max_n = n
            max_d = d
print(max_score, max_n, max_d)

0.5483618093786418 600 4


In [28]:
n_estimators = [50, 100, 150]

max_score = 0
for n in n_estimators:
    clf = BalancedRandomForestClassifier(n_estimators=n)
    result = cv_score(clf, Xtr_one_hot.values, Ytrain, cv=5)
    if (result[3] > max_score):
        max_score = result[3]
        max_n = n
print(max_score, max_n)

0.5459190328442887 150


Given the performance of the two chosen models, we end up with choosing the gradient boosting classifier (600 for n_estimators, and 4 for max_depth), with the leave-one-out scheme for encoding the categorical variables. 

## Performance of Selected Model 

We now train the chosen model with its parameters on the full training set and then test the model on the test set.

In [39]:
# Define the classifier and the under-sampler
clf = GradientBoostingClassifier(n_estimators=600, max_depth=4)
rus = RandomUnderSampler(random_state=0)

# Undersample the data
x_train, y_train = rus.fit_resample(Xtrain, Ytrain)

# Encode the data using leave-one-out Encoder
encoder = ce.leave_one_out.LeaveOneOutEncoder(
    cols=col_names).fit(x_train, y_train)
x_trans = encoder.transform(x_train)
x_test = encoder.transform(Xtest)
# Fit the model
clf.fit(x_trans, y_train)
# Test the model on the test set
ypred = clf.predict(x_test)
print('Accuracy:', accuracy(ypred, Ytest))
print("Precision:", precision(ypred, Ytest))
print("Recall:", recall(ypred, Ytest))
print("F1:", f1(ypred, Ytest))

Accuracy: 0.7469983981659735
Precision: 0.6966476415403792
Recall: 0.4574985998018181
F1: 0.5522962500650128


The final model performed as expected with 74.6% for accuracy and 0.55 for F1-score. 

Better performance can be achieved by including more information about the crashes, especially those related to the driver. Although the city of Chicago has online information about the vehicle and people involved in the crashes, but those data are mostly empty. 

There is one last issue that we would like to address; one of the important variable in determining the crash severity is the feature: primary contributory cause. We can check its importance by checking the feature importance returned by the gradient boosting classifier.

In [45]:
feat_imp = clf.feature_importances_
print("Features' Importance\n")
for col, importance in zip(X.columns[1:], feat_imp):
    print(col, importance)

Features' Importance

POSTED_SPEED_LIMIT 0.04119838996558089
TRAFFIC_CONTROL_DEVICE 0.015797754325966862
DEVICE_CONDITION 0.005238857016511208
WEATHER_CONDITION 0.005892923958717377
LIGHTING_CONDITION 0.03309817459354405
FIRST_CRASH_TYPE 0.4179544513315709
TRAFFICWAY_TYPE 0.0706627047691068
ALIGNMENT 0.006248160551500281
ROADWAY_SURFACE_COND 0.005795432123793122
PRIM_CONTRIBUTORY_CAUSE 0.2866866011158217
CRASH_HOUR 0.042036840553199076
CRASH_DAY_OF_WEEK 0.0066533316136133235
CRASH_MONTH 0.008567669160784499
AREAS 0.05416870892028976


Similarly to what we obtained with the Cramer'V coefficient, the two most important features are: "first_crash_type" (i.e., type of collision) and "prim_contributory_cause". Note that the latter feature ("prim_contributory_cause") has around 41% of its entries as "unable to determine" or "not applicable", which we primarily decided to keep them. (All entries of the feature "first_crash_type" are known).

In [30]:
unknown = (X.PRIM_CONTRIBUTORY_CAUSE.isin(
    ['UNABLE TO DETERMINE', 'NOT APPLICABLE']))
print("The percentage of undetermined entries for the primary cause variable is:",
      100*sum(unknown)/X.shape[0])

The percentage of undetermined entries for the primary cause variable is: 41.50697171341908


It would be interesting to look at the performance of the predictive model if we only focus on the known entries of the variable "Primary Contributory Cause". This is what we are going to check next.

## Focus on the Known Primary Contributory Cause

Let us first discard the unknown causes.

In [31]:
unknown = (X.PRIM_CONTRIBUTORY_CAUSE.isin(
    ['UNABLE TO DETERMINE', 'NOT APPLICABLE']))
X = X.drop(index=X.index[unknown])
Y = Y[~unknown]

We then split the data into training and test sets.

In [32]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X.iloc[:, 1:], Y, train_size=0.8, random_state=48)

We now train the final model we have chosen: gradient boosting classifier and fine-tune it on this subset of the data. 

In [33]:
n_estimators = [300, 500, 600]
max_depth = [3, 4, 5]

max_score = 0
for n in n_estimators:
    for d in max_depth:
        clf = GradientBoostingClassifier(n_estimators=n, max_depth=d)
        result = cv_score_LOO_undersample(
            clf, Xtrain, Ytrain, colnames=col_names, cv=5)
        if (result[3] > max_score):
            max_score = result[3]
            max_n = n
            max_d = d
print(max_score, max_n, max_d)

0.584158578921545 600 4


Let us now check its final performance on the test set.

In [34]:
# Define the classifier and the under-sampler
clf = GradientBoostingClassifier(n_estimators=600, max_depth=4)
rus = RandomUnderSampler(random_state=0)

# Undersample the data
x_train, y_train = rus.fit_resample(Xtrain, Ytrain)

# Encode the data using leave-one-out Encoder
encoder = ce.leave_one_out.LeaveOneOutEncoder(
    cols=col_names).fit(x_train, y_train)
x_trans = encoder.transform(x_train)
x_test = encoder.transform(Xtest)
# Fit the model
clf.fit(x_trans, y_train)
# Test the model on the test set
ypred = clf.predict(x_test)
print('Accuracy:', accuracy(ypred, Ytest))
print("Precision:", precision(ypred, Ytest))
print("Recall:", recall(ypred, Ytest))
print("F1:", f1(ypred, Ytest))

Accuracy: 0.7331356932894505
Precision: 0.701985508610144
Recall: 0.500167616493463
F1: 0.5841359329731423


We notice that we have a boost in the performance in terms of recall and F1-score, when we only focus on the known entries of the feature "prim_contributory_cause".

We conclude that better performance can be achieved by including more information about the crashes, especially those related to the driver. Although the city of Chicago has online information about the vehicle and people involved in the crashes, but those data are mostly empty.